In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, time, warnings, gzip, gc, random, math, shap, pickle, optuna, csv, sys
from IPython.display import display
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 400)

warnings.filterwarnings("ignore")

In [54]:
time0 = time.time()

with open('../input/lendingclub0720-downsampled57/LCLoans_57_800k.pkl', 'rb') as pickled_one:
    df = pickle.load(pickled_one)

df.drop(columns = ['pymnt_plan', 'fico_range_low'],inplace=True)
# fico bucket is always 5 points increment, so I drop lower bound as redundant info
    
df = df.sample(400000)
display(df.shape, time.time()-time0, df.head())

(400000, 55)

0.6274611949920654

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
150986101,38150,38150,38150.0,60,16.40,935.869995,C,C4,driver,10+ years,MORTGAGE,157000.0,Source Verified,Apr-2019,Current,debt_consolidation,Debt consolidation,743xx,OK,19.700001,0.0,Jun-1998,719,0.0,NaN,NaN,11.0,0.0,36570,72.4,34.0,w,33265.531250,33265.531250,10259.809570,10259.809570,4884.470215,5375.339844,0.0,0.0,0.0,May-2020,0.00000,Jul-2020,May-2020,749,745,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
78149002,4800,4800,4800.0,36,7.89,150.179993,A,A5,nursing tech,10+ years,MORTGAGE,38000.0,Source Verified,May-2016,Fully Paid,debt_consolidation,Debt consolidation,537xx,WI,10.390000,4.0,Feb-1990,759,1.0,3.0,NaN,14.0,0.0,36,0.1,20.0,w,0.000000,0.000000,5422.981934,5422.979980,4800.000000,622.979980,0.0,0.0,0.0,Jun-2019,4.61000,NaN,May-2020,799,795,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
153914191,30000,30000,30000.0,60,11.02,652.580017,B,B2,Teacher,10+ years,MORTGAGE,50000.0,Not Verified,Jun-2019,Current,debt_consolidation,Debt consolidation,853xx,AZ,29.670000,0.0,Aug-2001,689,0.0,NaN,NaN,16.0,0.0,30298,68.7,25.0,w,26548.070312,26548.070312,5854.850098,5854.850098,3451.929932,2402.919922,0.0,0.0,0.0,Jun-2020,0.00000,Aug-2020,May-2020,724,720,0.0,NaN,1,Joint App,120000.0,19.49,NaN,0.0
29974658,2450,2450,2450.0,36,15.61,85.669998,D,D1,Driver,5 years,RENT,25000.0,Not Verified,Oct-2014,Fully Paid,debt_consolidation,Debt consolidation,114xx,NY,3.600000,0.0,Jul-2008,749,0.0,NaN,NaN,4.0,0.0,1824,13.5,8.0,f,0.000000,0.000000,2901.080078,2901.080078,2450.000000,451.079987,0.0,0.0,0.0,Mar-2016,1532.47998,NaN,Mar-2016,659,655,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
137367662,7000,7000,7000.0,36,13.56,237.759995,C,C1,Clerk,10+ years,MORTGAGE,60000.0,Verified,Jul-2018,Current,debt_consolidation,Debt consolidation,462xx,IN,20.200001,0.0,Jul-2013,709,1.0,NaN,65.0,11.0,1.0,6270,23.7,15.0,w,3263.270020,3263.270020,4987.689941,4987.689941,3736.729980,1250.959961,0.0,0.0,0.0,May-2020,0.00000,Jun-2020,May-2020,729,725,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0


In [55]:
display(sys.getsizeof(df)/1048576)

103.23419570922852

### EDA

In [56]:
display(df.loan_status.value_counts())
df.target=np.nan
df.loc[df.loan_status.isin(['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']), 'target']=0
df.loc[df.loan_status.isin(['Charged Off', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off', 'Default']), 'target']=1
df=df[df['target'].isin([0,1])]
display(df.shape,df.loan_status.value_counts(), df.count())

Fully Paid                                             204843
Current                                                140790
Charged Off                                             49695
Late (31-120 days)                                       2224
In Grace Period                                          1388
Late (16-30 days)                                         363
Does not meet the credit policy. Status:Fully Paid        273
Issued                                                    271
Does not meet the credit policy. Status:Charged Off        90
Default                                                    63
Name: loan_status, dtype: int64

(257188, 56)

Fully Paid                                             204843
Charged Off                                             49695
Late (31-120 days)                                       2224
Does not meet the credit policy. Status:Fully Paid        273
Does not meet the credit policy. Status:Charged Off        90
Default                                                    63
Current                                                     0
In Grace Period                                             0
Issued                                                      0
Late (16-30 days)                                           0
Name: loan_status, dtype: int64

loan_amnt                      257188
funded_amnt                    257188
funded_amnt_inv                257188
term                           257188
int_rate                       257188
installment                    257188
grade                          257188
sub_grade                      257188
emp_title                      239023
emp_length                     241109
home_ownership                 257188
annual_inc                     257188
verification_status            257188
issue_d                        257188
loan_status                    257188
purpose                        257188
title                          254259
zip_code                       257188
addr_state                     257188
dti                            257030
delinq_2yrs                    257186
earliest_cr_line               257186
fico_range_high                257188
inq_last_6mths                 257186
mths_since_last_delinq         126459
mths_since_last_record          42597
open_acc    

In [57]:
display(df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv']].describe())
# basically, the three features are identical

display(df.term.value_counts())

,loan_amnt,funded_amnt,funded_amnt_inv
count,257188.000000,257188.000000,257188.000000
mean,14584.223409,14578.140407,14557.643555
std,8979.866868,8977.366128,8980.487305
min,500.000000,500.000000,0.000000
25%,7875.000000,7850.000000,7800.000000
50%,12000.000000,12000.000000,12000.000000
75%,20000.000000,20000.000000,20000.000000
max,40000.000000,40000.000000,40000.000000


36    192517
60     64671
Name: term, dtype: int64

In [58]:
# add key loan features

df['lti']=df.loan_amnt/df.annual_inc
df['iti']=(df.installment*12)/df.annual_inc





In [59]:
df.purpose.value_counts()

debt_consolidation    146784
credit_card            57164
home_improvement       17217
other                  15844
major_purchase          5828
medical                 3035
small_business          2851
car                     2674
vacation                1896
moving                  1830
house                   1523
wedding                  316
renewable_energy         161
educational               65
Name: purpose, dtype: int64

In [60]:
df.title.value_counts()

Debt consolidation                 129934
Credit card refinancing             50702
Home improvement                    15401
Other                               14258
Major purchase                       5074
                                    ...  
LU Tuition                              0
LUCK                                    0
LUCKY                                   0
La Colina Home Improvement Loan         0
îîMY FIRST CAR îî               0
Name: title, Length: 22589, dtype: int64

In [61]:
display(df.loc[df.target==1,'delinq_2yrs'].describe())
display(df.loc[df.target==0,'delinq_2yrs'].describe())
# i am not sure how this variable is defined. Is it at the moment of loan origination?

count    52072.000000
mean         0.332386
std          0.919772
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         36.000000
Name: delinq_2yrs, dtype: float64

count    205114.000000
mean          0.306771
std           0.861446
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          39.000000
Name: delinq_2yrs, dtype: float64

In [62]:
#df[['fico_range_low', 'fico_range_high']].drop_duplicates().sort_values(['fico_range_low'])

In [63]:
time.time()-time0

1.0815131664276123